In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  13.98 MiB
  allocs estimate:  219099
  --------------
  minimum time:     15.918 ms (0.00% GC)
  median time:      18.018 ms (0.00% GC)
  mean time:        20.704 ms (13.78% GC)
  maximum time:     32.029 ms (38.84% GC)
  --------------
  samples:          242
  evals/sample:     1

In [6]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9635032101229569, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=1.9635032101229573, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=2.9635032101229566, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=3.963503210122956, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=4.963503210122956, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.9635032101229575, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=6.963503210122957, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=7.963503210122956, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=8.963503210122953, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=9.963503210122955, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.963503210122955, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float